In [ ]:
#H03T05
#suurimmat kokonaispäästöt maittain

In [68]:
def emissions(url):
    import pandas as pd
    
    #tehdään taas frame huonosti muotoillusta tiedostosta
    df = pd.read_csv(url, sep='\t', engine='python')
    sarake = 'country\tcountry_code\tETS information\tmain activity sector name\tunit\tvalue\tyear'
    splittaus = df[sarake].str.split('\t').str
    uusi_df_pohja = {'country_code' : splittaus[1], 'main activity sector': splittaus[3], 'value' : splittaus[5], 'year' : splittaus[6]}
    df1 = pd.DataFrame(uusi_df_pohja)
    
    #poistetaan halutut rivit
    df1.drop(df1.loc[df1['main activity sector'] == '20-99 All stationary installations'].index, inplace=True)
    df1.drop(df1.loc[df1['main activity sector'] == '21-99 All industrial installations (excl. combustion)'].index, inplace=True)
    
    #poimitaan uuteen frameen halutut vuodet
    #aluksi sarake täytyy muuttaa numeeriseksi että vertailu onnistuu
    df1['year'] = pd.to_numeric(df1['year'], errors = 'coerce')
    df2 = df1.loc[df1['year'] >= 2010]
    df2 = df2.loc[df1['year'] <= 2015]
    
    #lasketaan eri maiden vuotuiset päästöt uuteen frameen
    df2['value'] = pd.to_numeric(df2['value'], errors = 'coerce')
    df3 = df2.groupby(['country_code','year'], as_index=False)['value'].sum()
    
    #poimitaan kunkin maan suurimman päästön vuosi ja määrä
    #sitten laitetaan saastuttajat suuruusjärjestykseen
    #helppoa ja hauskaa
    paastomax = df3.groupby(['country_code'])['value'].idxmax()
    df4 = df3.loc[paastomax]
    df4.sort_values('value', ascending=False, inplace=True)
    
    #vaadittua tulostusta varten nimetään sarake uudelleen
    #ja numeroarvot automaattitarkastuksen haluamaan muotoon
    df4.rename(columns = {'country_code': 'ccode'}, inplace=True)
    df4['year'] = df4['year'].astype(int)
    df4['value'] = df4['value'].map('{:e}'.format)
    
    #muotoillaan lopullinen tuloste
    tulos = df4[['ccode','year','value']].head(5)
    
    return tulos

In [69]:
#url="https://student.labranet.jamk.fi/~huoptu/datananal_s20/kerta3/harjoitukset/ETS_Database_v38A.csv"
url="https://student.labranet.jamk.fi/~huoptu/datananal_s20/kerta3/harjoitukset/ETS_Database_v38B.csv"
print(emissions(url))

    ccode  year         value
31     DE  2011  1.837570e+09
68     GB  2012  9.973063e+08
104    IT  2012  7.604445e+08
151    PL  2011  7.557359e+08
62     FR  2012  3.456367e+08
